In [9]:
import tensorflow as tf
import numpy as np

from tensorflow import keras
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.models import Sequential
# from tensorflow.keras.utils import np_utils
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras import layers 
import tensorflow.keras.backend as K 

from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler


import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
# import plotly.express as px
# import plotly.graph_objects as go

from math import sqrt

In [2]:
lstm_train_X = pd.read_csv("ERA\\ERA_train_x.csv")
lstm_train_y = pd.read_csv("ERA\\ERA_train_y.csv")

lstm_test_X = pd.read_csv("ERA\\ERA_test_x.csv")
lstm_test_y = pd.read_csv("ERA\\ERA_test_y.csv")

In [3]:
team = list(lstm_train_X.T_ID.unique())
year = list(lstm_train_y.YEAR.unique())

In [4]:
team, year

(['LG', 'HH', 'NC', 'HT', 'SK', 'KT', 'WO', 'LT', 'SS', 'OB'],
 [2016, 2017, 2018, 2019])

In [43]:
model_dict = dict()
hist_dict = dict()
test_pred_df = pd.DataFrame([],columns = ['YEAR','T_ID','y','y_pred','MSE','MSE_avg'])

idx = 0
for y in year:
    tmp1 = lstm_train_X[lstm_train_X["YEAR"] == y]
    tmp2 = lstm_train_y[lstm_train_y["YEAR"] == y]
    tmp3 = lstm_test_X[lstm_test_X["YEAR"] == y]
    tmp4 = lstm_test_y[lstm_test_y["YEAR"] == y]
    for t in team:
        name = '{}{}'.format(y,t)
        print(name,"=======================================")
        
        X_train = tmp1[tmp1["T_ID"] == t].drop(["T_ID","YEAR"],axis = 1)
        y_train = tmp2[tmp2["T_ID"] == t].drop(["T_ID","YEAR"],axis=1)
        X_test = tmp3[tmp3["T_ID"] == t].drop(["T_ID","YEAR"],axis=1)
        y_test = tmp4[tmp4["T_ID"] == t].drop(["T_ID","YEAR"],axis=1)
        
        X_train_v = X_train.values
        y_train_v = y_train.values

        X_test_v = X_test.values
        y_test_v = y_test.values
        """
        X_train_t = X_train_v.reshape(X_train_v.shape[0], 2,X_train_v.shape[1]//2)
        X_test_t = X_test_v.reshape(X_test_v.shape[0], 2,X_test_v.shape[1]//2)
        """
        
        ## model
        K.clear_session() 
        
        model = Sequential()
        optimizer = Adam(lr=0.01)
        model.add(Dense(1024, input_shape = [X_train.shape[1]]))
        model.add(Dense(1024))
        model.add(Dense(1024))
        model.add(Dense(1024))
        model.add(Dense(1024))
        model.add(Dense(1)) # output = 1
        model.compile(loss='mean_squared_error', optimizer=optimizer,metrics=['mae'])

       
        model.summary()
        
        early_stop = EarlyStopping(monitor='loss', mode = 'min',patience=2, verbose=1)

        hist1 = model.fit(X_train, y_train, epochs=100,
                  batch_size=1, verbose=1, callbacks=[early_stop])
        ##
        
        model_dict[name] = model
        hist_dict[name] = hist1
        
        y_pred = model.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        
        ######### MSE AVG#########
        mse_avg = mean_squared_error(y_test,[y_train.mean()[0]])
        
        
        test_pred_df.loc[idx,:] = [y,t,y_test_v.reshape(-1)[0],y_pred.reshape(-1)[0], mse,mse_avg]

        idx += 1

test_pred_df[['y','y_pred',"shift_AVG_1","shift_AVG_2",'MSE','MSE_avg']] = test_pred_df[['y','y_pred',"shift_AVG_1","shift_AVG_2",'MSE','MSE_avg']].astype(float)

W0923 15:50:01.512622 18616 training.py:504] Falling back from v2 loop because of error: Failed to find data adapter that can handle input: <class 'pandas.core.frame.DataFrame'>, <class 'NoneType'>


2016LG =======================================
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1024)              48128     
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 1025      
Total params: 2,148,353
Trainable params: 2,148,353
Non-trainable params: 0
_________________________________________________________________
Train on 73 samples
Epoch 1/100
73/73 [==============================] - 1s 10ms/sample - loss: 338.3643 - mae: 2.3521
Epoch 2/100
73/73 [==============================] - 1s 9ms/sample - loss: 0.0

W0923 15:50:09.290825 18616 training.py:504] Falling back from v2 loop because of error: Failed to find data adapter that can handle input: <class 'pandas.core.frame.DataFrame'>, <class 'NoneType'>


Epoch 00011: early stopping
2016HH =======================================


W0923 15:50:09.523206 18616 training.py:504] Falling back from v2 loop because of error: Failed to find data adapter that can handle input: <class 'pandas.core.frame.DataFrame'>, <class 'NoneType'>


...

W0923 15:54:26.470953 18616 training.py:504] Falling back from v2 loop because of error: Failed to find data adapter that can handle input: <class 'pandas.core.frame.DataFrame'>, <class 'NoneType'>


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1024)              48128     
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 1025      
Total params: 2,148,353
Trainable params: 2,148,353
Non-trainable params: 0
_________________________________________________________________
Train on 73 samples
Epoch 1/100
73/73 [==============================] - 1s 9ms/sample - loss: 308.8204 - mae: 2.3120
Epoch 2/100
73/73 [==============================] - 1s 9ms/sample - loss: 9.8467e-04 - mae: 0.0252
Epoch 3/100
73/73 [=======

W0923 15:54:31.751833 18616 training.py:504] Falling back from v2 loop because of error: Failed to find data adapter that can handle input: <class 'pandas.core.frame.DataFrame'>, <class 'NoneType'>


Epoch 00007: early stopping


KeyError: "['shift_AVG_1', 'shift_AVG_2'] not in index"

In [56]:
from sklearn.metrics import r2_score
tmp = test_pred_df.copy()
# tmp['half']= 0.5
r2_y_predict = r2_score(tmp['y'], tmp['y_pred'])
r2_y_predict

-1.3222042274068913